In [1]:
import numpy as np
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import process
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.metrics import DistanceMetric
import matplotlib.pyplot as plt
from spellchecker import SpellChecker
from sklearn.metrics import pairwise_distances

C:\Users\pratik\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
import pandas as pd

# Load the Excel file
df = pd.read_excel('Event_date.xlsx')  

# Combine 'Primary Category' and 'Description' into a new 'Combined' column
df['Combined'] = df['Location'].astype(str) + " : " + df['Primary Category'].astype(str)+" : " + df['Sub Category'].astype(str)+" : " + \
df['Event Names'].astype(str) + " : " + df['Description'].astype(str)
df['Location_Combined'] = df['Location'].astype(str) + " : " + df['Description'].astype(str)
# Check the DataFrame after adding the 'Combined' column
print("\nDataFrame with Combined Column:")
df.head()


DataFrame with Combined Column:


,Event ID,Event Names,Description,Primary Category,Secondary Category,Sub Category,Location,Prices,Date,Unnamed: 9,Time,Links,Combined,Location_Combined
0,EVT0001,Food Photography Workshop,Calling all food lovers and aspiring photograp...,Photography,NaN,Food Photography,Wanowrie,2500,2024-03-31,Sunday,10:00:00,https://allevents.in/pune/food-photography-wor...,Wanowrie : Photography : Food Photography : Fo...,Wanowrie : Calling all food lovers and aspirin...
1,EVT0002,Street Photography Workshop,Ten days of shooting includes two street photo...,Photography,NaN,Travel Photography,Dhayari,1200,2024-03-24,Sunday,07:00:00,NaN,Dhayari : Photography : Travel Photography : S...,Dhayari : Ten days of shooting includes two st...
2,EVT0003,Ajay Atul Live,Let's enjoy the magic of Ajay-Atul’s music lik...,Music and Entertainment,NaN,Concert,Bavdhan,1500,2024-04-20,Saturday,19:00:00,https://in.bookmyshow.com/events/ajay-atul-liv...,Bavdhan : Music and Entertainment : Concert : ...,Bavdhan : Let's enjoy the magic of Ajay-Atul’s...
3,EVT0005,Poetry and Comedy,"GOTU LIT WEDNESDAY'S, A COMEDY AND POETRY OPEN...",Music and Entertainment,NaN,"Poetry,Comedy",Viman Nagar,400,2024-03-24,Sunday,17:00:00,NaN,Viman Nagar : Music and Entertainment : Poetry...,"Viman Nagar : GOTU LIT WEDNESDAY'S, A COMEDY A..."
4,EVT0006,Global Education Fair,🌟 Why Attend the Fair? One to One Interaction ...,Books and Education,NaN,Academic,Dhankawadi,0,2024-04-13,Saturday,11:00:00,https://allevents.in/pune/%F0%9F%8E%93-global-...,Dhankawadi : Books and Education : Academic : ...,Dhankawadi : 🌟 Why Attend the Fair? One to One...


In [4]:
query= input("Enter your Query: ")

Enter your Query:  events at shivajinagar


In [5]:
import re
from spellchecker import SpellChecker
from fuzzywuzzy import process

def correct_query(query, exclusion_list):
    spell = SpellChecker()
    
    # Normalize and tokenize the query while keeping punctuations
    tokens = re.findall(r"[\w'-]+|[.,!?;]", query)
    
    # Process tokens to handle multi-word exclusions and corrections
    corrected_tokens = []
    skip_next = False
    for i, token in enumerate(tokens):
        if skip_next:
            skip_next = False
            continue


        combined_token = token
        if i + 1 < len(tokens) and tokens[i + 1] not in ".,!?;":
            combined_token += " " + tokens[i + 1]

        # Find the best match from the exclusion list
        match_score = process.extractOne(combined_token, exclusion_list, score_cutoff=80)
        
        if match_score:
            match, score = match_score
            # If a good match is found, use it and skip the next token if it was part of the match
            corrected_tokens.append(match)
            if " " in combined_token:
                skip_next = True
        else:
            # If no match, process normally
            corrected_token = spell.correction(token.lower())
            if token[0].isupper():  # Preserve capitalization
                corrected_token = corrected_token.capitalize()
            corrected_tokens.append(corrected_token)

    # Join tokens, handle spaces around punctuation
    corrected_query = ''.join([' '+i if not i.startswith("'") and i not in ".,!?;" else i for i in corrected_tokens]).strip()
    return corrected_query

# Example usage
known_locations = df['Location'].dropna().unique().tolist()  # Proper names to the list
#query = "concrt shiva nagr"
corrected_query = correct_query(query, known_locations)

print("Original Query:", query)
print("Corrected Query:", corrected_query)

Original Query: events at shivajinagar
Corrected Query: events Shivajinagar


In [6]:
from datetime import datetime, timedelta
 
# Example data loading
# Assuming df is defined somewhere in your code
 
def get_date_range(query):
    today = datetime.today()
    if 'today' in query:
        start_date = end_date = today
    elif 'tomorrow' in query:
        start_date = end_date = today + timedelta(days=1)
    elif 'day after tomorrow' in query:
        start_date = end_date = today + timedelta(days=2)
    elif 'this weekend' in query:
        start_date = today + timedelta((5 - today.weekday()) % 7) 
        end_date = start_date + timedelta(days=1)  
    elif 'next weekend' in query:
        start_date = today + timedelta((12 - today.weekday())) 
        end_date = start_date + timedelta(days=1)  
    elif 'next month' in query:
        first_next_month = today.replace(day=1) + timedelta(days=31)
        start_date = first_next_month.replace(day=1)
        end_date = start_date.replace(month=start_date.month % 12 + 1, day=1) - timedelta(days=1)
    else:
        start_date = min(df['Date'])
        end_date = max(df['Date'])
    return start_date, end_date
 
def filter_events(query):
    start_date, end_date = get_date_range(query)
    filtered_df = df[(df['Date'] >= start_date.strftime('%Y-%m-%d')) & (df['Date'] <= end_date.strftime('%Y-%m-%d'))]
    return filtered_df
 
filtered_df = filter_events(corrected_query)

In [7]:
filtered_df

,Event ID,Event Names,Description,Primary Category,Secondary Category,Sub Category,Location,Prices,Date,Unnamed: 9,Time,Links,Combined,Location_Combined
0,EVT0001,Food Photography Workshop,Calling all food lovers and aspiring photograp...,Photography,NaN,Food Photography,Wanowrie,2500,2024-03-31,Sunday,10:00:00,https://allevents.in/pune/food-photography-wor...,Wanowrie : Photography : Food Photography : Fo...,Wanowrie : Calling all food lovers and aspirin...
1,EVT0002,Street Photography Workshop,Ten days of shooting includes two street photo...,Photography,NaN,Travel Photography,Dhayari,1200,2024-03-24,Sunday,07:00:00,NaN,Dhayari : Photography : Travel Photography : S...,Dhayari : Ten days of shooting includes two st...
2,EVT0003,Ajay Atul Live,Let's enjoy the magic of Ajay-Atul’s music lik...,Music and Entertainment,NaN,Concert,Bavdhan,1500,2024-04-20,Saturday,19:00:00,https://in.bookmyshow.com/events/ajay-atul-liv...,Bavdhan : Music and Entertainment : Concert : ...,Bavdhan : Let's enjoy the magic of Ajay-Atul’s...
3,EVT0005,Poetry and Comedy,"GOTU LIT WEDNESDAY'S, A COMEDY AND POETRY OPEN...",Music and Entertainment,NaN,"Poetry,Comedy",Viman Nagar,400,2024-03-24,Sunday,17:00:00,NaN,Viman Nagar : Music and Entertainment : Poetry...,"Viman Nagar : GOTU LIT WEDNESDAY'S, A COMEDY A..."
4,EVT0006,Global Education Fair,🌟 Why Attend the Fair? One to One Interaction ...,Books and Education,NaN,Academic,Dhankawadi,0,2024-04-13,Saturday,11:00:00,https://allevents.in/pune/%F0%9F%8E%93-global-...,Dhankawadi : Books and Education : Academic : ...,Dhankawadi : 🌟 Why Attend the Fair? One to One...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,EVT0197,Almost Human,@almosthumanofc Currently Biggest Name In INDI...,Music and Entertainment,NaN,Concert,Hadapsar,500,2024-04-07,NaN,21:00:00,https://insider.in/almost-human-live-pune-tech...,Hadapsar : Music and Entertainment : Concert :...,Hadapsar : @almosthumanofc Currently Biggest N...
165,EVT0198,Sky Jumper Trampoline Park - Pune\n,Discover the extraordinary Indoor Amusement Pa...,Music and Entertainment,NaN,NaN,Yerawada,700,2024-04-28,NaN,NaN,NaN,Yerawada : Music and Entertainment : nan : Sky...,Yerawada : Discover the extraordinary Indoor A...
166,EVT0199,Day Trip at Girivan - private hill station,NaN,Travel,NaN,Camping,NaN,580,2024-04-21,NaN,07:00:00,https://in.bookmyshow.com/activities/day-trip-...,nan : Travel : Camping : Day Trip at Girivan -...,nan : nan
167,EVT0200,THE LIT BUG FEST 2024 SEASON 6\n,Experience literature from all facets,Art and Culture,Music and Entertainment,NaN,Baner,700,2024-04-20,NaN,10:00:00,https://in.bookmyshow.com/activities/the-lit-b...,Baner : Art and Culture : nan : THE LIT BUG FE...,Baner : Experience literature from all facets


In [8]:
model = SentenceTransformer("all-MiniLM-L6-v2")

# Embedding vectors
embedding_arr = model.encode(filtered_df['Combined'].tolist())

In [9]:
query_embedding = model.encode(query)
# Distance computation
dist = DistanceMetric.get_metric('euclidean')
dist_arr = dist.pairwise(embedding_arr, query_embedding.reshape(1, -1)).flatten()
# Sort results
idist_arr_sorted = np.argsort(dist_arr)
 
filtered_df.iloc[idist_arr_sorted]

,Event ID,Event Names,Description,Primary Category,Secondary Category,Sub Category,Location,Prices,Date,Unnamed: 9,Time,Links,Combined,Location_Combined
20,EVT0027,Gift and Craft Expo,NaN,Art and Culture,NaN,"Craft,Exhibition",Shivajinagar,0,2024-07-11,Thursday,07:00:00,NaN,"Shivajinagar : Art and Culture : Craft,Exhibit...",Shivajinagar : nan
33,EVT0040,Chess Festival,"This Summer, enjoy chess by participating in o...",Sports,NaN,Chess,Shivajinagar,300,2024-09-27,Friday,18:00:00,https://allevents.in/pune/lbhm-chess-festival-...,Shivajinagar : Sports : Chess : Chess Festival...,"Shivajinagar : This Summer, enjoy chess by par..."
38,EVT0045,Velas Turtle Festival,Velas Turtle Festival and Raigad with Sahyadri...,Travel,NaN,Camping,Shivajinagar,2000,2024-09-19,Thursday,18:00:00,https://allevents.in/pimpri%20chinchwad/velas-...,Shivajinagar : Travel : Camping : Velas Turtle...,Shivajinagar : Velas Turtle Festival and Raiga...
163,EVT0196,Omen 3: Carniva of Chaos,Enter the dark immersive art gallery of Omen C...,Music and Entertainment,NaN,Concert,Shivajinagar,700,2024-04-07,NaN,15:33:00,https://in.bookmyshow.com/events/omen-iii-a-ca...,Shivajinagar : Music and Entertainment : Conce...,Shivajinagar : Enter the dark immersive art ga...
92,EVT0110,Knife Painting Workshop,Lets learn to make your paintings 3D using san...,Art and Culture,NaN,Drawing and Painting,Shivajinagar,Free,2024-05-05,NaN,18:00:00,NaN,Shivajinagar : Art and Culture : Drawing and P...,Shivajinagar : Lets learn to make your paintin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,EVT0137,Unlocking Opportunities: Integrating Generativ...,Discover the essential skills needed to thrive...,Tech and Innovation,NaN,Artificial Intelligence,Camp,0,2024-04-22,NaN,11:00:00,NaN,Camp : Tech and Innovation : Artificial Intell...,Camp : Discover the essential skills needed to...
94,EVT0113,Time Throttle,Time Throttle is a progressive rock band that ...,Music and Entertainment,NaN,Concert,Camp,299,2024-05-05,NaN,21:00:00,NaN,Camp : Music and Entertainment : Concert : Tim...,Camp : Time Throttle is a progressive rock ban...
125,EVT0148,Learn English Punctuation,Knowing how and when to use punctuation correc...,Books and Education,NaN,Academic,Lohegaon,1400,2024-04-30,NaN,10:00:00,NaN,Lohegaon : Books and Education : Academic : Le...,Lohegaon : Knowing how and when to use punctua...
129,EVT0152,Improv Comedy Workshop 101,Ready to unlock your potential and take your s...,Music and Entertainment,NaN,Comedy,Aundh,2999,2024-04-18,NaN,19:00:00,NaN,Aundh : Music and Entertainment : Comedy : Imp...,Aundh : Ready to unlock your potential and tak...


### Code Flow and Explanation:

1. **Import Libraries**:
   - `numpy` and `pandas` for data manipulation and numerical operations.
   - `fuzzywuzzy` for fuzzy string matching.
   - `SentenceTransformer` from `sentence_transformers` for generating sentence embeddings.
   - `PCA` from `sklearn.decomposition` for dimensionality reduction (optional, not used here).
   - `DistanceMetric` from `sklearn.metrics` for computing pairwise distances between embeddings.
   - `SpellChecker` for correcting spelling errors in the user query.
   - `re` for regular expression matching.
   - `matplotlib` for plotting (optional, not used here).

2. **Load and Prepare Data**:
   - Load event data from `Event_date.xlsx`.
   - Combine relevant columns (`Location`, `Primary Category`, `Sub Category`, `Event Names`, `Description`) into a single column `Combined` for further analysis.
   - This combined column serves as the input for text-based similarity searches.

3. **User Query Handling and Correction**:
   - The `correct_query()` function processes the user query:
     - It corrects misspelled words using the `SpellChecker` and handles multi-word matches using `fuzzywuzzy`.
     - It compares the query tokens to known locations (from the dataset) to provide context-specific corrections.

4. **Date Range Parsing**:
   - The `get_date_range()` function extracts a date range based on specific keywords (e.g., "today", "this weekend", "next month").
   - This allows filtering the events by date based on the user query.

5. **Event Filtering by Date**:
   - The `filter_events()` function filters events from the dataset that occur within the identified date range, extracted from the query.
   - This step limits the scope of the search to relevant events based on time constraints in the user query.

6. **Text Embedding Using SentenceTransformer**:
   - A pre-trained model, `all-MiniLM-L6-v2` from **SentenceTransformer**, is used to convert event descriptions and the user query into dense numerical vectors (embeddings).
   - The embeddings capture the semantic meaning of the event descriptions and user query for similarity analysis.

7. **Distance Calculation**:
   - The `DistanceMetric.get_metric('euclidean')` computes the Euclidean distance between the query embedding and the event description embeddings.
   - Events are sorted by proximity to the query embedding, with closer (more relevant) events ranked higher.

8. **Result Sorting and Output**:
   - The events are sorted by their distance from the user query, with the most relevant events displayed at the top of the list.
   - The resulting dataframe contains the most relevant events based on both semantic similarity and date filtering.

### Example of Usage:
- The user enters a query, for example: `"concert near shiva nagar this weekend"`.
- The system corrects the query, extracts the relevant date range (e.g., "this weekend"), and filters events that match both the date and semantic content of the query.
- Finally, it ranks the events based on semantic similarity using SentenceTransformer embeddings.


